In [14]:
import prettytable
import sqlite3
import pandas as pd
import prettytable
prettytable.DEFAULT = 'DEFAULT'
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [12]:
connection = sqlite3.connect('db_sandbox.db')
%sql sqlite:///db_sandbox.db

### Problem 1
Find the total number of crimes recorded in the CRIME table.

In [17]:
%sql SELECT COUNT(*) FROM ChicagoCrimeData

 * sqlite:///db_sandbox.db
Done.


COUNT(*)
533


### Problem 2
List community area names and numbers with per capita income less than 11000.

In [24]:
%%sql 
SELECT * FROM PRAGMA_TABLE_INFO('socioeconomicChicago') 
    where name like '%capita%'
    OR name like '%name%'
    OR name like '%number%'

 * sqlite:///db_sandbox.db
Done.


cid,name,type,notnull,dflt_value,pk
0,COMMUNITY_AREA_NUMBER,REAL,0,None,0
1,COMMUNITY_AREA_NAME,TEXT,0,None,0
7,PER_CAPITA_INCOME,INTEGER,0,None,0


In [25]:
%%sql 
SELECT 
    COMMUNITY_AREA_NAME,
    COMMUNITY_AREA_NUMBER, 
    PER_CAPITA_INCOME
FROM socioeconomicChicago
WHERE PER_CAPITA_INCOME < 11000


 * sqlite:///db_sandbox.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER,PER_CAPITA_INCOME
West Garfield Park,26.0,10934
South Lawndale,30.0,10402
Fuller Park,37.0,10432
Riverdale,54.0,8201


### Problem 3
List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [32]:
%%sql 
SELECT * FROM PRAGMA_TABLE_INFO('chicagoCrimeData') LIMIT 5

 * sqlite:///db_sandbox.db
Done.


cid,name,type,notnull,dflt_value,pk
0,ID,INTEGER,0,None,0
1,CASE_NUMBER,VARCHAR(50),0,None,0
2,DATE,NVARCHAR(50),0,None,0
3,BLOCK,NVARCHAR(50),0,None,0
4,IUCR,INTEGER,0,None,0


In [37]:
%%sql 
SELECT 
    *
FROM chicagoCrimeData
WHERE UPPER(DESCRIPTION) LIKE '%MINOR%'
OR UPPER(PRIMARY_TYPE) LIKE '%MINOR%'

 * sqlite:///db_sandbox.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
3987219,HL266884,2005-03-31,024XX N CLARK ST,2210,LIQUOR LAW VIOLATION,SELL/GIVE/DEL LIQUOR TO MINOR,CONVENIENCE STORE,TRUE,FALSE,2333,19,43,7,22,1172680,1916483,2005,41.92626872,-87.64089934,"(41.926268719, -87.640899336)"
3266814,HK238408,2004-03-13,093XX S STONY ISLAND AVE,2230,LIQUOR LAW VIOLATION,ILLEGAL CONSUMPTION BY MINOR,ALLEY,TRUE,FALSE,413,4,8,48,22,1188539,1843379,2004,41.72530099,-87.58496589,"(41.72530099, -87.584965887)"


### Problem 4
List all kidnapping crimes involving a child?

In [35]:
%%sql 
SELECT 
    *
FROM chicagoCrimeData
WHERE 
    UPPER(DESCRIPTION) LIKE '%CHILD%'
    AND UPPER(PRIMARY_TYPE) LIKE '%KIDNAP%'

 * sqlite:///db_sandbox.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,FALSE,FALSE,1533,15,29,25,20,1143050,1897546,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


### Problem 5
List the kind of crimes that were recorded at schools. (No repetitions)

In [41]:
%%sql 
SELECT DISTINCT
    PRIMARY_TYPE
FROM chicagoCrimeData
WHERE 
    LOCATION_DESCRIPTION LIKE '%SCHOOL%'

 * sqlite:///db_sandbox.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


### Problem 6
List the type of schools along with the average safety score for each type.

In [44]:
%%sql 
SELECT * FROM PRAGMA_TABLE_INFO('ChicagoPublicSchools') 
    where name like '%safety%'
    OR name like '%name%'
    OR name like '%school%'

 * sqlite:///db_sandbox.db
Done.


cid,name,type,notnull,dflt_value,pk
0,School_ID,INTEGER,0,None,0
1,NAME_OF_SCHOOL,VARCHAR(64),0,None,0
2,"Elementary, Middle, or High School",VARCHAR(50),0,None,0
10,Collaborative_Name,VARCHAR(50),0,None,0
15,HEALTHY_SCHOOL_CERTIFIED,VARCHAR(50),0,None,0
16,Safety_Icon,VARCHAR(50),0,None,0
17,SAFETY_SCORE,INTEGER,0,None,0
74,COMMUNITY_AREA_NAME,VARCHAR(50),0,None,0


In [78]:
%%sql 
SELECT 
    `Elementary, Middle, or High School` AS SCHOOL_TYPE,
    ROUND(AVG(SAFETY_SCORE),3) AS AVG_SAFETY_SCORE

FROM ChicagoPublicSchools
GROUP BY SCHOOL_TYPE

 * sqlite:///db_sandbox.db
Done.


SCHOOL_TYPE,AVG_SAFETY_SCORE
ES,44.697
HS,45.355
MS,48.0


### Problem 7
List 5 community areas with highest % of households below poverty line

In [52]:
%%sql 
SELECT * FROM PRAGMA_TABLE_INFO('socioeconomicChicago') 
    where name like '%number%'
    OR name like '%name%'
    OR name like '%poverty%'

 * sqlite:///db_sandbox.db
Done.


cid,name,type,notnull,dflt_value,pk
0,COMMUNITY_AREA_NUMBER,REAL,0,None,0
1,COMMUNITY_AREA_NAME,TEXT,0,None,0
3,PERCENT_HOUSEHOLDS_BELOW_POVERTY,REAL,0,None,0


In [54]:
%%sql 
SELECT 
    COMMUNITY_AREA_NUMBER,
    COMMUNITY_AREA_NAME,
    PERCENT_HOUSEHOLDS_BELOW_POVERTY
FROM socioeconomicChicago
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC
LIMIT 5

 * sqlite:///db_sandbox.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
54.0,Riverdale,56.5
37.0,Fuller Park,51.2
68.0,Englewood,46.6
29.0,North Lawndale,43.1
27.0,East Garfield Park,42.4


### Problem 8
Which community area is most crime prone? Display the coumminty area number only.

In [66]:
%%sql
SELECT
    COMMUNITY_AREA_NUMBER,
    CRIME_NUMBER

FROM (
    SELECT 
        COMMUNITY_AREA_NUMBER,
        COUNT(COMMUNITY_AREA_NUMBER) AS CRIME_NUMBER
    FROM ChicagoCrimeData
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY CRIME_NUMBER DESC
) LIMIT 6

 * sqlite:///db_sandbox.db
Done.


COMMUNITY_AREA_NUMBER,CRIME_NUMBER
,43
25,43
23,22
68,21
29,16
28,16


### Problem 9
Use a sub-query to find the name of the community area with highest hardship index

In [68]:
%%sql 
SELECT * FROM PRAGMA_TABLE_INFO('socioeconomicChicago') 
    where name like '%hardship%'
    or name like '%community%'

 * sqlite:///db_sandbox.db
Done.


cid,name,type,notnull,dflt_value,pk
0,COMMUNITY_AREA_NUMBER,REAL,0,None,0
1,COMMUNITY_AREA_NAME,TEXT,0,None,0
8,HARDSHIP_INDEX,REAL,0,None,0


In [69]:
%%sql
SELECT 
    COMMUNITY_AREA_NAME
FROM (
    SELECT 
        COMMUNITY_AREA_NUMBER,
        COMMUNITY_AREA_NAME,
        HARDSHIP_INDEX
    FROM socioeconomicChicago
    ORDER BY HARDSHIP_INDEX DESC
) LIMIT 1

 * sqlite:///db_sandbox.db
Done.


COMMUNITY_AREA_NAME
Riverdale


### Problem 10
Use a sub-query to determine the Community Area Name with most number of crimes?

In [76]:
%%sql
SELECT
    crime.COMMUNITY_AREA_NUMBER,
    socio.COMMUNITY_AREA_NAME,
    crime.CRIME_NUMBER

FROM (
    SELECT 
        COMMUNITY_AREA_NUMBER,
        COUNT(COMMUNITY_AREA_NUMBER) AS CRIME_NUMBER
    FROM ChicagoCrimeData
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY CRIME_NUMBER DESC
) AS crime
INNER JOIN socioeconomicChicago AS socio
ON crime.COMMUNITY_AREA_NUMBER = socio.COMMUNITY_AREA_NUMBER
ORDER BY crime.CRIME_NUMBER DESC


 * sqlite:///db_sandbox.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,CRIME_NUMBER
25,Austin,43
23,Humboldt park,22
68,Englewood,21
28,Near West Side,16
29,North Lawndale,16
8,Near North Side,15
71,Auburn Gresham,14
24,West Town,13
66,Chicago Lawn,12
67,West Englewood,12
